In [1]:
import os, sys
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew

load_dotenv(find_dotenv())

True

In [2]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)


In [3]:
from payslip_processor import PayslipProcessor

payslip = PayslipProcessor(r"N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub.pdf")

Tables already extracted with Nanonets
Table 1 saved to N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\Table_1.csv
Table 2 saved to N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\Table_2.csv
Table 3 saved to N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\Table_3.csv
Table 4 saved to N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\Table_4.csv
Table 5 saved to N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\Table_5.csv
Table 6 saved to N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysha

In [4]:
payslip.table_paths

['N:\\Dev\\AI\\Underwriting\\data\\clients\\Oysharja Sayed and Saadman Ahmed\\Oysharja\\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\\Table_1.csv',
 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Oysharja Sayed and Saadman Ahmed\\Oysharja\\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\\Table_2.csv',
 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Oysharja Sayed and Saadman Ahmed\\Oysharja\\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\\Table_3.csv',
 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Oysharja Sayed and Saadman Ahmed\\Oysharja\\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\\Table_4.csv',
 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Oysharja Sayed and Saadman Ahmed\\Oysharja\\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\\Table_5.csv',
 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Oysharja Sayed and Saadman Ahmed\\Oysharja\\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_ta

In [5]:
from crewai_tools import BaseTool, CSVSearchTool

csv_tools = []
for csv in payslip.table_paths:
    csv_tools.append(CSVSearchTool(csv=csv))

class TableProcessingTool(BaseTool):
    name: str = "Table processing tool"
    description: str = "This tool "

    def _run(self, argument: str) -> str:
        # Implementation goes here
        return "Result from custom tool"

In [6]:
from langchain_openai import ChatOpenAI
from crewai import Agent
model = ChatOpenAI(
    model="gpt-4-turbo-preview"
)
csv_agent = Agent(
        role='Table data extractor',
        goal='Query and extract data from a structured table from in CSV format',
        tools=csv_tools,
        backstory='''As a Table data extractor, your mission is to extract relevant information from a CSV table. 
        The data in the table was extracted using an OCR and the headers might not be accurate.
        You must do your best to derive the correct information from tables that might not be perfect''',
        verbose=True,
        allow_delegation=False,
        llm=model,
        step_callback=lambda x: print_agent_output(x,"Senior Research Analyst Agent"),
    )
# def table_text_agent(self):
#         return Agent(
#             role='Table text data extractor',
#             goal='Extract data from an unstructured table in CSV format',
#             backstory='''As a Table text data extractor, your mission is to derive relevant information from a table formatted as CSV. 
#             The information you gather will be compared to that of the specialized structured CSV table data extractor agent.
#             In addition to that, you will also be tasked with finding out information which was not included in the structured tables or the specialized structured CSV table data extractor agent could not find''',
#             verbose=True,
#         )

In [7]:
from crewai import Task

get_salary_task = Task(
            description='''Given the tables from the employee's payslip analyze them and return the salary of the employee''',
            expected_output="A numerical value of the salary\n\neg. If the empolyee's salary is 70,0000USD the output will be: 70,000",
            agent=csv_agent,
        )

In [8]:
from crewai import Crew, Process

# Create Crew responsible for Copy
crew = Crew(
	agents=[csv_agent],
	tasks=[get_salary_task],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [9]:
result = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(result)

 [DEBUG]: == Working Agent: Table data extractor
 [INFO]: == Starting Task: Given the tables from the employee's payslip analyze them and return the salary of the employee


> Entering new CrewAgentExecutor chain...
Thought: To determine the salary of the employee from the payslips, I need to find the table that contains salary information. This information could be in terms like "Gross Pay," "Monthly Salary," "Annual Salary," or simply "Salary." Since the headers might not be accurate, I should start by searching for these terms across the tables to locate the relevant data.

Action: Search a CSV's content

Action Input: {"search_query": "Gross Pay"}


Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


Successfully saved N:\Dev\AI\Underwriting\data\clients\Oysharja Sayed and Saadman Ahmed\Oysharja\BRXM-F011215_Oysharja_Sayed_2023_Year_End_Paystub extracted_tables\Table_1.csv (DataType.CSV). New chunks count: 4
 

Relevant Content:
TAXES: Employment Insurance, Current: 0.00, YTD: 1,002.45, :

: , STATEMENT: , OF
BULLETIN DE: , EARNINGS
PAIE: 2.591,88

: , STATEMENT: , OF
BULLETIN DE: , EARNINGS
PAIE:

Thought: The observation indicates a "Gross Pay" or earnings figure, but it's not immediately clear which table this is from or if it directly provides a salary amount. The amount "2.591,88" might be relevant but it's unclear if this is a monthly, bi-weekly, or annual figure due to formatting and lack of context. I should search for additional terms that could clarify the salary amount, such as "Annual Salary" or "Monthly Salary," to ensure I have a comprehensive understanding.

Action: Search a CSV's content

Action Input: {"search_query": "Annual Salary"}
Doc content has not changed. S